In [22]:
pip cache purge


Files removed: 840 (2433.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [24]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.4/184.4 MB 11.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 11.1 MB/s eta 0:00:00a 0:00:01


In [25]:
pip install ultralytics


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 123.5 kB/s eta 0:00:00a 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 125.1 kB/s eta 0:00:0000:0200:14
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 133.0 kB/s eta 0:00:0000:0300:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.7/772.7 kB 132.2 kB/s eta 0:00:00a 0:00:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 MB 145.9 kB/s eta 0:00:0000:0200:08
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.2
    Uninstalling numpy-2.3.2:
      Successfully uninstalled numpy-2.3.2
Note: you may need to restart the kernel to use updated packages.


Importing the necessary Libraries

In [43]:
import os
import pandas as pd
from PIL import Image
import random
import shutil
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ultralytics import YOLO


In [47]:
# ==========================
# 🔧 Caminhos principais
# ==========================
base_dir = "/home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR"
images_dir = os.path.join(base_dir, "dataset")                   # imagens RGB baixadas
labels_csv = os.path.join(base_dir, "matched_galaxies_1000.csv") # CSV com classes morfológicas
dataset_cls = os.path.join(base_dir, "dataset_cls")              # novo dataset p/ classificação

# ==========================
# 🗂️ Criar estrutura padrão
# ==========================
for split in ["train", "val"]:
    for cls in ["smooth", "disk", "artifact"]:
        os.makedirs(os.path.join(dataset_cls, split, cls), exist_ok=True)

# ==========================
# 📄 Carregar dados e limpar
# ==========================
df = pd.read_csv(labels_csv)
df = df.dropna(subset=["morph_class"]).drop_duplicates(subset=["RA", "DEC"])
df = df.reset_index(drop=True)

# Mapear caminhos das imagens correspondentes
def image_path(idx):
    fname = f"galaxy_{idx:04d}.png"
    path = os.path.join(images_dir, fname)
    return path if os.path.exists(path) else None

df["image_path"] = [image_path(i) for i in range(len(df))]
df = df.dropna(subset=["image_path"])

# Dividir dataset (80% treino / 20% validação)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["morph_class"])

# ==========================
# 📦 Copiar imagens por classe
# ==========================
def copy_images(subset_df, subset_name):
    for _, row in subset_df.iterrows():
        cls = row["morph_class"]
        src = row["image_path"]
        dst = os.path.join(dataset_cls, subset_name, cls, os.path.basename(src))
        shutil.copy(src, dst)

copy_images(train_df, "train")
copy_images(val_df, "val")

print(f"✅ Total de treino: {len(train_df)} | validação: {len(val_df)}")
print(f"📁 Estrutura pronta em: {dataset_cls}")

# ==========================
# 🚀 Treinar modelo CLS
# ==========================
model = YOLO("yolov8n-cls.pt")  # modelo base de classificação (pré-treinado no ImageNet)

results = model.train(
    data=dataset_cls,  # YOLO detecta automaticamente a estrutura
    epochs=30,         # aumente para 50-100 para resultados melhores
    imgsz=128,         # imagens pequenas (suficiente para textura galáctica)
    batch=16,
    device="cpu",      # use "cuda" se tiver GPU disponível
    project="treinamento_cls",
    name="modelo_cls_galaxias"
)

print("\n🏁 Treinamento concluído com sucesso!")

# ==========================
# 📊 Avaliação e métricas
# ==========================
metrics = model.val()
print("\n📈 Métricas de classificação:")
for k, v in metrics.results_dict.items():
    print(f"{k}: {v:.4f}")

# ==========================
# 📉 Curvas de acurácia
# ==========================
if hasattr(results, "metrics") and "accuracy_top1" in results.metrics:
    plt.figure(figsize=(8, 5))
    plt.plot(results.metrics["accuracy_top1"], label="Top-1 Accuracy")
    plt.plot(results.metrics["accuracy_top5"], label="Top-5 Accuracy")
    plt.title("Curvas de acurácia - YOLOv8 CLS")
    plt.xlabel("Épocas")
    plt.ylabel("Acurácia")
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("⚠️ Métricas detalhadas não disponíveis diretamente — veja logs em /runs/classify/")

# ==========================
# 🔬 Testar o modelo (ajustado - opção 1)
# ==========================

# Caminho para a pasta de validação com subpastas por classe
test_dir = os.path.join(dataset_cls, "val")

# O curinga */* faz o YOLO buscar imagens dentro de todas as subpastas de classe
preds = model.predict(source=f"{test_dir}/*/*", conf=0.5, save=True)

print(f"\n🔭 Resultados salvos em: {preds[0].save_dir}")


✅ Total de treino: 874 | validação: 219
📁 Estrutura pronta em: /home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR/dataset_cls
New https://pypi.org/project/ultralytics/8.3.217 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.216 🚀 Python-3.12.9 torch-2.9.0+cpu CPU (AMD Ryzen 5 4600G with Radeon Graphics)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR/dataset_cls, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=128, int8=False, iou=0.7, keras=False, kobj=1.0, line_wid

In [50]:
# ==============================
# 📈 Avaliação e exportação final + Curvas de aprendizado (corrigido)
# ==============================

from ultralytics import YOLO
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

# Caminho do modelo treinado
model_path = "/home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR/treinamento_cls/modelo_cls_galaxias3/weights/best.pt"
model = YOLO(model_path)

# Avaliar desempenho no conjunto de validação
results = model.val(split='val')

print("\n📊 Métricas principais:")
print(f"Top-1 Accuracy: {results.top1:.4f}")
print(f"Top-5 Accuracy: {results.top5:.4f}")

# ==============================
# 🔹 Matriz de confusão
# ==============================
if hasattr(results, "confusion_matrix") and results.confusion_matrix.matrix is not None:
    cm = results.confusion_matrix.matrix
    fig, ax = plt.subplots(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt=".1f", cmap="coolwarm", cbar=False,
                xticklabels=['smooth', 'disk', 'artifact'],
                yticklabels=['smooth', 'disk', 'artifact'])
    plt.xlabel("Predito")
    plt.ylabel("Verdadeiro")
    plt.title("Matriz de confusão - Classificação de Galáxias")
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Matriz de confusão não disponível nesta versão do Ultralytics.")

# ==============================
# 🔹 Curvas de aprendizado
# ==============================
train_results_path = "/home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR/treinamento_cls/modelo_cls_galaxias3/results.csv"

if os.path.exists(train_results_path):
    df_results = pd.read_csv(train_results_path)
    
    plt.figure(figsize=(10, 5))
    plt.plot(df_results["epoch"], df_results["train/loss"], label="Train Loss", linewidth=2)
    if "metrics/accuracy_top1" in df_results.columns:
        plt.plot(df_results["epoch"], df_results["metrics/accuracy_top1"], label="Accuracy (Top-1)", linewidth=2)
    plt.title("Evolução do Treinamento - Loss e Acurácia")
    plt.xlabel("Época")
    plt.ylabel("Valor")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Arquivo de resultados não encontrado. Verifique se o treino salvou o results.csv.")

# ==============================
# 💾 Exportar modelo treinado (corrigido)
# ==============================
export_dir = "/home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR/modelos_exportados"
os.makedirs(export_dir, exist_ok=True)

print("\n💾 Exportando modelos...")
# O modelo já está em .pt (PyTorch), exportamos só os demais formatos
model.export(format="onnx", imgsz=128, dynamic=False, simplify=True)
model.export(format="torchscript", imgsz=128, dynamic=False)
model.export(format="openvino", imgsz=128, dynamic=False)

print(f"✅ Modelos exportados com sucesso em: {export_dir}")


Ultralytics 8.3.216 🚀 Python-3.12.9 torch-2.9.0+cpu CPU (AMD Ryzen 5 4600G with Radeon Graphics)
YOLOv8n-cls summary (fused): 30 layers, 1,438,723 parameters, 0 gradients, 3.3 GFLOPs
train: /home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR/dataset_cls/train... found 874 images in 3 classes ✅ 
val: /home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR/dataset_cls/val... found 219 images in 3 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3561.4±994.8 MB/s, size: 233.6 KB)
val: Scanning /home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR/dataset_cls/val... 219 images, 0 corrupt: 100% ━━━━━━━━━━━━ 219/219 584.7Kit/s 0.0s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 14/14 14.0it/s 1.0s0.2s
                   all      0.598          1
Speed: 0.0ms preprocess, 1.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR/runs/classify/val6

📊 Métricas principais:

<Figure size 600x500 with 1 Axes>

<Figure size 1000x500 with 1 Axes>


💾 Exportando modelos...
Ultralytics 8.3.216 🚀 Python-3.12.9 torch-2.9.0+cpu CPU (AMD Ryzen 5 4600G with Radeon Graphics)

PyTorch: starting from '/home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR/treinamento_cls/modelo_cls_galaxias3/weights/best.pt' with input shape (1, 3, 128, 128) BCHW and output shape(s) (1, 3) (2.8 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.71', 'onnxruntime'] not found, attempting AutoUpdate...
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/18.2 MB ? eta -:--:--
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/18.2 MB ? eta -:--:--
   ━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/18.2 MB 7.7 MB/s eta 0:00:03
   ━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/18.2 MB 8.4 MB/s eta 0:00:02
   ━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/18.2 MB 10.7 MB/s eta 0:00:02
   ━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 9.4/18.2 MB 10.9 MB/s eta 0:00:01
   ━━━━━━━━━

2025-10-17 18:26:09.884212679 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card0/device/vendor"


ONNX: export success ✅ 15.9s, saved as '/home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR/treinamento_cls/modelo_cls_galaxias3/weights/best.onnx' (5.5 MB)

Export complete (16.0s)
Results saved to /home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR/treinamento_cls/modelo_cls_galaxias3/weights
Predict:         yolo predict task=classify model=/home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR/treinamento_cls/modelo_cls_galaxias3/weights/best.onnx imgsz=128  
Validate:        yolo val task=classify model=/home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR/treinamento_cls/modelo_cls_galaxias3/weights/best.onnx imgsz=128 data=/home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR/dataset_cls  
Visualize:       https://netron.app
Ultralytics 8.3.216 🚀 Python-3.12.9 torch-2.9.0+cpu CPU (AMD Ryzen 5 4600G with Radeon Graphics)

PyTorch: starting from '/home/astroesul/Asaf_mestrado/Códigos_atuais/Hardware-BR/treinamento_cls/modelo_cls_galaxias3/weights/best.pt' with in